In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import pandas as pd

URL = 'https://comic.naver.com/webtoon/weekday.nhn'
data = requests.get(URL).text #테그 제외하고 text만 출력하는 부분
soup = BeautifulSoup(data,'html.parser') #html.parser로 html 형식으로 불러옵니다.

In [3]:
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>네이버 만화 &gt; 요일별  웹툰 &gt; 전체웹툰</title>
<meta content="네이버 웹툰" property="og:title"/>
<meta content="https://ssl.pstatic.net/static/comic/images/og_tag_v2.png" property="og:image"/>
<meta content="매일매일 새로운 재미, 네이버 웹툰." property="og:description"/>
<meta content="https://comic.naver.com/webtoon/weekday" property="og:url"/>
<meta content="article" property="og:type"/>
<meta content="네이버 웹툰" property="og:article:author"/>
<meta content="https://comic.naver.com" property="og:article:author:url"/>
<link href="https://ssl.pstatic.net/static/comic/favicon/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<script type="text/javascript">
		document.domain = "naver.com";
	</script>
<link href="/css/comic/comic_20210818143918.css" rel="stylesheet" style="text/css"/>
<!-- comicWeekdayJsFile -->
<script c

In [8]:
title = soup.find_all('a', {'class' : 'title'}) # <a class = "title">인 html소스를 찾아 title에 전부 추가

#찾고 싶은 리스트는 제목,작가,요일,장르,줄거리,플렛폼(나중에 카카오페이지도 포함할 수도 있기 때문에)
id_list = []; title_list = []; author_list = []; day_list = []; genre_list = []; story_list = []; platform_list = [];
num = 0

In [10]:
driver = webdriver.Chrome("C:/chromedriver/chromedriver.exe")
driver.get(URL)

#crome 화면 열어서 crawling 진행하는 부분, 따로 driver설치하는 법 설명

In [11]:
for i in range(len(title)):
    sleep(0.5) # 크롤링 중간 중간 텀을 주어 과부하 생기지 않도록

    page = driver.find_elements_by_class_name("title")
    page[i].click() # 요일당 첫번째 웹툰 차례로 클릭하기

    sleep(0.5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') # 이동한 페이지 주소 html.parser

    day = soup.find_all('ul', {'class' : 'category_tab'})
    day = day[0].find('li', {'class' : 'on'}).text[0:1] # 요일 모아

    t = title[i].text #title문서에서 text만 뽑아오기
    if (t in title_list):  # 요일 두 개 이상이면 요일만 추가함
        day_list[title_list.index(t)] += ', ' + day
        driver.back()
        continue
        #같은 웹툰이기 때문에 다시 진행할 이유가 없음

    id_list.append(num) ; num += 1  # id에 추가
    title_list.append(t)  # 제목 리스트에 추가
    day_list.append(day) # 요일 리스트에 추가
    platform_list.append('네이버 웹툰') # 플랫폼 리스트에 추가

    author = soup.find_all('h2') # h2태그에 있음
    author = author[1].find('span', {'class' : 'wrt_nm'}).text[8:] # 8번 째부터 작가 성함
    author_list.append(author) # 작가에 추가

    genre = soup.find('span', {'class' : 'genre'}).text # 장르 부분
    a = genre.index(",")
    genre = genre[a+1:].replace(" ","") #앞부분 스토리, 옴니버스, 드라마는 필요하지 않아서 사용하지 않음
    genre_list.append(genre) # 장르에 추가

    story = soup.find_all('p') # 줄거리 수집
    story = str(story[3])
    story = story.replace('<p>', '').replace('</p>', '').replace('<br/>', '\n') # <br>을 개행으로 바꾸기
    story_list.append(story) # 줄거리에 추가 #<br> tag는 특수문자 제거로 바꾸기

    driver.back() # 뒤로 가기

[<selenium.webdriver.remote.webelement.WebElement (session="0a13d1caa54ce1273370ccc0e6e1514c", element="75000012-5827-4cf7-a4f1-7fcb64510b2b")>, <selenium.webdriver.remote.webelement.WebElement (session="0a13d1caa54ce1273370ccc0e6e1514c", element="f191cac1-ef95-49bf-8965-d1f57a30a806")>, <selenium.webdriver.remote.webelement.WebElement (session="0a13d1caa54ce1273370ccc0e6e1514c", element="216f0831-5263-4073-bb8d-59725e0bd2af")>, <selenium.webdriver.remote.webelement.WebElement (session="0a13d1caa54ce1273370ccc0e6e1514c", element="6665e10c-e818-4155-889f-d3396dd007c2")>, <selenium.webdriver.remote.webelement.WebElement (session="0a13d1caa54ce1273370ccc0e6e1514c", element="e6482954-093d-4801-968b-cc8c2a1608db")>, <selenium.webdriver.remote.webelement.WebElement (session="0a13d1caa54ce1273370ccc0e6e1514c", element="e4d2f275-e6ad-4b9e-ad2c-08d0cea3b34c")>, <selenium.webdriver.remote.webelement.WebElement (session="0a13d1caa54ce1273370ccc0e6e1514c", element="3f7eaa30-b7d8-487d-95fb-5eea1193

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=92.0.4515.159)


In [5]:
total_data = pd.DataFrame()
total_data['id'] = id_list
total_data['title'] = title_list
total_data['author'] = author_list
total_data['day'] = day_list
total_data['genre'] = genre_list
total_data['story'] = story_list
total_data['platform'] = platform_list

In [6]:
total_data
#크롤링 결과, 네이버_웹툰_2인 이유는 genre를 변경했기 때문에

,id,title,author,day,genre,story,platform
0,0,신의 탑,SIU,월,판타지,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년\n그리고 그런 소년을 시험하는 탑,네이버 웹툰
1,1,참교육,채용택 / 한가람,월,액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!\n&lt;부...,네이버 웹툰
2,2,뷰티풀 군바리,설이 / 윤성원,월,드라마,'여자도 군대에 간다면?'본격 여자도 군대 가는 만화!,네이버 웹툰
3,3,윈드브레이커,조용석,월,스포츠,혼자서 자전거를 즐겨타던 모범생 조자현.\n원치 않게 자전거 크루의 일에 자꾸 휘말...,네이버 웹툰
4,4,장씨세가 호위무사,조형근 / 김인호,월,액션,‘당신이 부른 것이오. 나란 사람을... ’\n은둔고수 광휘. 호위무사 되다.\n웹...,네이버 웹툰
...,...,...,...,...,...,...,...
435,435,제타,하지,일,스릴러,극심한 슬럼프에 안드로이드 '제타'의 그림을 표절해버린 금세기 최고의 천재 화가 '...,네이버 웹툰
436,436,푸른불꽃,닺,일,드라마,유년시절 트라우마를 겪은 '문가야'. 새롭게 전학 온 학교에서 완벽한 모범생이자 모...,네이버 웹툰
437,437,데이즈,양수,일,스릴러,대한민국 최고 재벌가의 첫째 아들이자 집안의 망나니 ‘한지한’은 오늘도\n거하게 취...,네이버 웹툰
438,438,샤인 스타,김현,일,로맨스,전학생 선유별이 좋아하는 김샤인은 학교에서 유명한 달리기 스타. 잘생긴 얼굴에 뛰어...,네이버 웹툰


In [7]:
total_data.to_csv('네이버_웹툰_2.csv', encoding = 'utf-8-sig')